This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [7]:
structure_name = "LLZO_010_Li_order4_off__Li_100_slab_heavy"

In [8]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_Li_order4_off__Li_100_slab_heavy
- Initial lattice height: 66.24 Å
- Number of atoms: 536


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 18.89 Å
Lowest Li slab atom: 32.35 Å
LLZO top z: 32.20 Å
Li penetration into LLZO: -0.15 Å


In [10]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 17.06 Å
Li slab thickness:   18.89 Å
LLZO top z:          32.20 Å
Lowest Li atom z:    32.35 Å
Li penetration into LLZO: -0.15 Å


In [11]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 399 atoms out of 536
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_171700/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [12]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 19:36:58    -2127.541367        0.753932
SciPyFminCG:    1 19:36:59    -2130.261938        5.285879
SciPyFminCG:    2 19:37:00    -2150.381439       11.064434
SciPyFminCG:    3 19:37:01    -2152.943676        8.950124
SciPyFminCG:    4 19:37:02    -2164.631821        4.299288
SciPyFminCG:    5 19:37:02    -2170.434616        3.289085
SciPyFminCG:    6 19:37:03    -2172.350990        1.972039
SciPyFminCG:    7 19:37:04    -2173.860218        0.923450
SciPyFminCG:    8 19:37:05    -2174.872845        2.975738
SciPyFminCG:    9 19:37:06    -2175.639343        1.835555
SciPyFminCG:   10 19:37:06    -2176.597275        1.088328
SciPyFminCG:   11 19:37:07    -2177.116879        0.749300
SciPyFminCG:   12 19:37:08    -2177.453228        0.544085
SciPyFminCG:   13 19:37:09    -2177.690666        1.032356
SciPyFminCG:   14 19:37:10    -2178.029060        2.126551
SciPyFminCG:   15 19:37:11    -2179.111206        1.503226

In [13]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_171700/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 19:38:08    -2255.363628        0.116494
FIRE:    1 19:38:08    -2255.366951        0.112770
FIRE:    2 19:38:09    -2255.372829        0.111122
FIRE:    3 19:38:09    -2255.380241        0.108142
FIRE:    4 19:38:10    -2255.387909        0.103352
FIRE:    5 19:38:10    -2255.395576        0.101287
FIRE:    6 19:38:10    -2255.403244        0.100172
FIRE:    7 19:38:10    -2255.411678        0.101501
FIRE:    8 19:38:11    -2255.421902        0.103909
FIRE:    9 19:38:11    -2255.435192        0.117228
FIRE:   10 19:38:12    -2255.452827        0.125083
FIRE:   11 19:38:12    -2255.475319        0.124796
FIRE:   12 19:38:12    -2255.502411        0.126446
FIRE:   13 19:38:13    -2255.534359        0.124905
FIRE:   14 19:38:13    -2255.569885        0.103372
FIRE:   15 19:38:14    -2255.606434        0.096608
FIRE:   16 19:38:14    -2255.641960        0.094555
FIRE:   17 19:38:14    -2255.675186        0.102115
FIRE:   18 19:

In [14]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [15]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 18.82 Å
Lowest Li slab atom: 32.42 Å
LLZO top z: 32.00 Å
Li penetration into LLZO: -0.42 Å


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 36 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [16]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 19:38:41    -2257.307606        1.069106
SciPyFminCG:    1 19:38:42    -2258.945904        0.872208
SciPyFminCG:    2 19:38:43    -2261.050133        1.517590
SciPyFminCG:    3 19:38:44    -2261.939823        0.968865
SciPyFminCG:    4 19:38:45    -2262.962673        0.851879
SciPyFminCG:    5 19:38:46    -2264.043541        1.256585
SciPyFminCG:    6 19:38:47    -2265.045177        0.822789
SciPyFminCG:    7 19:38:47    -2265.706375        0.824148
SciPyFminCG:    8 19:38:48    -2266.478497        1.169444
SciPyFminCG:    9 19:38:50    -2267.282055        1.205840
SciPyFminCG:   10 19:38:51    -2268.868469        1.770387
SciPyFminCG:   11 19:38:51    -2271.513260        0.876319
SciPyFminCG:   12 19:38:52    -2273.114243        0.772880
SciPyFminCG:   13 19:38:53    -2273.769051        0.749649
SciPyFminCG:   14 19:38:54    -2274.173130        0.749570
SciPyFminCG:   15 19:38:54    -2274.416191        0.712261

In [17]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_171700/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 19:40:57    -2355.418911        0.126483
FIRE:    1 19:40:57    -2355.419933        0.126412
FIRE:    2 19:40:58    -2355.421722        0.126279
FIRE:    3 19:40:58    -2355.424023        0.126104
FIRE:    4 19:40:58    -2355.426323        0.125922
FIRE:    5 19:40:58    -2355.428623        0.125763
FIRE:    6 19:40:59    -2355.430923        0.125648
FIRE:    7 19:40:59    -2355.433735        0.125585
FIRE:    8 19:40:59    -2355.436802        0.125563
FIRE:    9 19:41:00    -2355.440380        0.125549
FIRE:   10 19:41:00    -2355.444214        0.125498
FIRE:   11 19:41:00    -2355.448814        0.125333
FIRE:   12 19:41:00    -2355.454182        0.124984
FIRE:   13 19:41:01    -2355.460571        0.124456
FIRE:   14 19:41:01    -2355.467983        0.123876
FIRE:   15 19:41:01    -2355.476929        0.123364
FIRE:   16 19:41:02    -2355.486385        0.122789
FIRE:   17 19:41:02    -2355.497631        0.121855
FIRE:   18 19:

In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_Li_order4_off__Li_100_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 